# Script for Processing Data

In [3]:
%load_ext autoreload
%autoreload 2

import os
import pandas as pd
import numpy as np
import random
import glob
import umap
import seaborn as sns
import matplotlib.pyplot as plt
import pickle
# from ggplot import *


from ruffus import *
import sys, os, h5py, random, tempfile, scipy, time,copy
import cmapPy.pandasGEXpress.parse_gctx as parse_gctx
import cmapPy.pandasGEXpress.parse_gct as parse_gct
import pipeline_support as PS
from sklearn.decomposition import PCA
from scipy import stats
from matplotlib.pyplot import imshow
from sklearn.preprocessing import MinMaxScaler

from collections import Counter
#from tensorflow_examples.models.pix2pix import pix2pix
import tensorflow as tf
from tensorflow import keras
from IPython.display import clear_output
from tensorflow_gan.python.losses import losses_impl


from numpy.random import seed
randomState = 123
seed(randomState)

Parameters

In [4]:
n_sampling = 50000

Input Filenames

In [5]:
l1000_filename = "../data/L1000/GSE92742_Broad_LINCS_Level3_INF_mlr12k_n1319138x12328.gctx"
l1000_geneinfo_filename = "../data/L1000/GSE92742_Broad_LINCS_gene_info.txt"
l1000_sampleinfo_filename = "../data/L1000/GSE92742_Broad_LINCS_inst_info.txt"

l1000_all_gene_list = "../data/L1000/all_gene_list.txt"
l1000_landmark_gene_list = "../data/L1000/landmark_gene_list.txt"

archs4_all_gene_list = "../data/ARCHS4/all_gene_list.txt"

gtex_l1000_all_gene_list = "../data/GTEx/l1000_all_gene_list.txt"
gtex_l1000_landmark_gene_list = "../data/GTEx/l1000_landmark_gene_list.txt"

gtex_rnaseq_all_gene_list = "../data/GTEx/rnaseq_all_gene_list.txt"

Output Filenames

In [6]:
l1000_output_filename = "../data/processed/L1000/L1000_filtered_GSE92742_Broad_LINCS_Level3_INF_mlr12k_n{}x{}.pkl" # only landmark genes
l1000_overlap_landmark_output_filename = "../data/processed/L1000/L1000_filtered_GSE92742_Broad_LINCS_Level3_INF_mlr12k_n{}x{}_cellline_{}.f" # n_samplingx967


## Get overlap_landmark_file

In [7]:
with open(l1000_landmark_gene_list, "r") as f:
    l1000_landmark_gene = [x.strip() for x in f.readlines()]
with open(archs4_all_gene_list, "r") as f:
    archs4_all_gene = [x.strip() for x in f.readlines()]
with open(gtex_l1000_landmark_gene_list, "r") as f:
    gtex_l1000_landmark_gene = [x.strip() for x in f.readlines()]
with open(gtex_rnaseq_all_gene_list, "r") as f:
    gtex_rnaseq_all_gene = [x.strip() for x in f.readlines()]
    



In [8]:
overlap_landmark_genes = list(set(l1000_landmark_gene).intersection(archs4_all_gene).intersection(gtex_l1000_landmark_gene).intersection(gtex_rnaseq_all_gene))

## Load L1000 (GSE92742/Level 3) ~4 min

https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE92742 downloaded @ ../data

In [11]:
print('Loading L1000 data.....')
l1000_data = parse_gctx.parse(l1000_filename,convert_neg_666=True).data_df

Loading L1000 data.....


In [12]:
gene_info = pd.read_csv(l1000_geneinfo_filename,header = 0, sep = '\t')

In [13]:
# create a probe_id to gene name dictionary 
gene_dict = dict(zip([str(x) for x in gene_info['pr_gene_id']], gene_info['pr_gene_symbol']))

# label rows with gene names 
l1000_data.index = [gene_dict[x] for x in l1000_data.index.values]


In [14]:
# filter landmark genes
filtered_l1000_data = l1000_data[l1000_data.index.isin(overlap_landmark_genes)]

In [15]:
# save filtered_l1000 data (only landmark genes)
with open(l1000_output_filename.format(filtered_l1000_data.shape[0], filtered_l1000_data.shape[1]), "wb") as f:
    pickle.dump(filtered_l1000_data, l1000_output_filename)
# filtered_l1000_data.to_csv(l1000_output_filename)

TypeError: dump() missing required argument 'file' (pos 2)

Remove variables

In [19]:
del l1000_data

NameError: name 'l1000_data' is not defined

In [11]:
sample_info = pd.read_csv(l1000_sampleinfo_filename, sep="\t")

In [12]:
cell_lines = sample_info["cell_id"].unique()

In [13]:
cell_lines

array(['MCF7', 'PC3', 'HA1E', 'HCC515', 'VCAP', 'A375', 'HEPG2', 'HT29',
       'A549', 'A673', 'AGS', 'CL34', 'CORL23', 'COV644', 'DV90', 'EFO27',
       'HCC15', 'HCT116', 'HEC108', 'HT115', 'JHUEM2', 'LOVO', 'MDST8',
       'NCIH1694', 'NCIH1836', 'NCIH2073', 'NCIH508', 'NCIH596', 'NOMO1',
       'OV7', 'PL21', 'RMGI', 'RMUGS', 'SKLU1', 'SKM1', 'SKMEL1',
       'SKMEL28', 'SNGM', 'SNU1040', 'SNUC4', 'SNUC5', 'SW480', 'SW620',
       'SW948', 'T3M10', 'THP1', 'TYKNU', 'U937', 'WSUDLCL2', 'ASC',
       'NEU', 'NPC', 'PHH', 'SKB', 'HUH7', 'SHSY5Y', 'HEK293T', 'SKL',
       'HS27A', 'BT20', 'HS578T', 'MCF10A', 'MDAMB231', 'SKBR3', 'NKDBA',
       'FIBRNPC', 'MCH58', 'NCIH716', 'CD34', 'H1299', 'HL60', 'JURKAT',
       'RKO', 'U266', 'HEKTE', 'U2OS'], dtype=object)

In [30]:
for cell_line in cell_lines:
    sample_ids_cell_line = sample_info.loc[sample_info["cell_id"] == cell_line, "inst_id"].tolist()

    if len(sample_ids_cell_line) > n_sampling:
        filtered_l1000_data_cell_line = filtered_l1000_data.loc[:, sample_ids_cell_line]

        filtered_l1000_data_cell_line_sampled = filtered_l1000_data_cell_line.sample(axis='columns', n=n_sampling).T
        filtered_l1000_data_cell_line_sampled.reset_index().to_feather(l1000_overlap_landmark_output_filename.format(filtered_l1000_data_cell_line_sampled.shape[0], filtered_l1000_data_cell_line_sampled.shape[1], cell_line))

        print(l1000_overlap_landmark_output_filename.format(filtered_l1000_data_cell_line_sampled.shape[0], filtered_l1000_data_cell_line_sampled.shape[1], cell_line))



MCF7
(962, 176765)
../data/processed/L1000/L1000_filtered_GSE92742_Broad_LINCS_Level3_INF_mlr12k_n50000x962_cellline_MCF7.f
PC3
(962, 178476)
../data/processed/L1000/L1000_filtered_GSE92742_Broad_LINCS_Level3_INF_mlr12k_n50000x962_cellline_PC3.f
HA1E
(962, 87902)
../data/processed/L1000/L1000_filtered_GSE92742_Broad_LINCS_Level3_INF_mlr12k_n50000x962_cellline_HA1E.f
HCC515
(962, 74683)
../data/processed/L1000/L1000_filtered_GSE92742_Broad_LINCS_Level3_INF_mlr12k_n50000x962_cellline_HCC515.f
VCAP
(962, 204383)
../data/processed/L1000/L1000_filtered_GSE92742_Broad_LINCS_Level3_INF_mlr12k_n50000x962_cellline_VCAP.f
A375
(962, 114727)
../data/processed/L1000/L1000_filtered_GSE92742_Broad_LINCS_Level3_INF_mlr12k_n50000x962_cellline_A375.f
HEPG2
(962, 66070)
../data/processed/L1000/L1000_filtered_GSE92742_Broad_LINCS_Level3_INF_mlr12k_n50000x962_cellline_HEPG2.f
HT29
(962, 113867)
../data/processed/L1000/L1000_filtered_GSE92742_Broad_LINCS_Level3_INF_mlr12k_n50000x962_cellline_HT29.f
A549
(9